Set Colab to GPU Mode if you are training the models!

### Mounting Google Drive

In [62]:
# from google.colab import drive
# drive.mount('/content/drive')

In [63]:
# cd 'drive/MyDrive/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

### Importing Necessary Libraries


In [64]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import os

### Importing Data
We import the data of a young subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC
- **Default-mode areas:** mPFC, ACC, PCC, Precuneus, mOFC
- **Sensory areas:** VC
- **Multisensory area:** pSTS
- **Some other areas:** TP, IPL, mCC, Put, PCG, Nac, INS





In [65]:
# file_names = [name for name in os.listdir('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in os.listdir('./Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in file_names if ("537" not in name)]

In [66]:
ROI_names = [file_name.replace("data_","").replace(".npy","") for file_name in file_names]

In [67]:
data_ori = list()
for file_name in file_names:
    # data_ori.append(np.load('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))
    data_ori.append(np.load('./Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))

### Data Preparation

Now, here we prepare the data to be fed to the Neural Network Architechtures. We can't predict the time ahead upto which the subjects are predicting correctly so, we will take a bit round about way to predict the BOLD values ahead in time(for several choices of time points e.g. 1 time point ahead, 2 time point ahead, etc) and will compare the results for young and old subjects.

Right from the starting we decide some of the data preparation parameters, so that we can use them as we require.

In [68]:
TRAIN_BATCH_SIZE = 20
TEST_BATCH_SIZE = 10
LOOK_AHEADS = list(range(1, 16)) 
NUM_TEST_SUBS = 10

We will start a basic data preparation where we will average across all the voxels for a ROI and fit the model which predicts this average BOLD value. Later we will avoid doing an average since, all voxels in a ROI is not equally important and hence we will want to predict the BOLD values of several voxels simultaneously.

In [69]:
data = [np.mean(dat, axis=1, keepdims=True) for dat in data_ori]

Let's Start the Data Preparation for lOFC later we will iterate the same procedure for the other ROIs.

In [70]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

TP Average BOLD Shape: (189, 1, 111)


Below, we see the average BOLD time series plot for all the ROIs.

We have changed the data in the form (Number of Subjects, TR, Voxels) below.

In [71]:
data = [np.transpose(dat, (2,0,1)) for dat in data]

In [72]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

TP Average BOLD Shape: (111, 189, 1)


In [73]:
# for i in range(len(data)):  
#   plt.figure(figsize=(10, 8))
#   plt.plot(data[i][0,:,0])
#   plt.title("Average BOLD of 1st Subject for "+ROI_names[i])
#   plt.xlabel("TRs")
#   plt.ylabel("Average BOLD(over voxels)")
#   plt.savefig("./Plots/YOUNG/AVG-BOLD/AVG-BOLD-"+ROI_names[i]+".png")

We split the dataset of 111 young subjects into 101 subjects in the training data and 10 subject in the test data.

In [74]:
train_data_unscaled = [dat[:-NUM_TEST_SUBS] for dat in data]
test_data_unscaled = [dat[-NUM_TEST_SUBS:] for dat in data]  

We scaled the BOLD values of each voxel in the train data between 0 and 1 for each voxel. We use the same MinMaxScaler fit on the train data on the test data.

In [75]:
scalers = list()
for i in range(len(ROI_names)):
  scalers.append(MinMaxScaler(feature_range = (0, 1)))
train_data = [sc.fit_transform(dat[:,:,0]) for sc, dat in zip(scalers, train_data_unscaled)] 
test_data = [sc.transform(dat[:,:,0]) for sc, dat in zip(scalers, test_data_unscaled)]

In [76]:
train_data = [np.expand_dims(dat, axis = -1) for dat in train_data]
test_data = [np.expand_dims(dat, axis = -1) for dat in test_data]

In [77]:
print(ROI_names[-1], "Train Data Shape: ", train_data[-1].shape)
print(ROI_names[-1], "Test Data Shape: ", test_data[-1].shape)

TP Train Data Shape:  (101, 189, 1)
TP Test Data Shape:  (10, 189, 1)


In [78]:
# for i in range(len(data)):  
#   plt.figure(figsize=(10, 8))
#   plt.plot(train_data[i][0,:,0])
#   plt.title("Average BOLD of 1st Subject for "+ROI_names[i])
#   plt.xlabel("TRs")
#   plt.ylabel("Average BOLD(over voxels)[Scaled between 0 and 1]")
#   plt.savefig("./Plots/YOUNG/AVG-BOLD/AVG-BOLD-SCALED-"+ROI_names[i]+".png")

We now build a data generator which would produce mini-batches during the training of the neural network.

In [79]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_TRs, batch_size, look_ahead=1):
        self.data = data
        self.num_TRs = num_TRs
        self.batch_size = batch_size
        self.current_idx = 0
        self.look_ahead = look_ahead

    def generate(self):
        while True:
            if self.current_idx + self.batch_size >= self.data.shape[0]:
              self.current_idx = 0
            x = self.data[self.current_idx:(self.current_idx+self.batch_size),:(-self.look_ahead),:]
            y = self.data[self.current_idx:(self.current_idx+self.batch_size),self.look_ahead:,:]
            self.current_idx = self.current_idx + self.batch_size
            yield x, y 

In [80]:
train_data_generator, test_data_generator = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    train_data_generator.append([KerasBatchGenerator(dat, dat.shape[1], TRAIN_BATCH_SIZE, LOOK_AHEAD) for dat in train_data])
    test_data_generator.append([KerasBatchGenerator(dat, dat.shape[1], TEST_BATCH_SIZE, LOOK_AHEAD) for dat in test_data])

In [81]:
batch_xs, batch_ys = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    batch = next(train_data_generator[LOOK_AHEAD-1][-1].generate())
    batch_xs.append(batch[0])
    batch_ys.append(batch[1])
    print(f"{ROI_names[-1]} Train Batch X Shape for {LOOK_AHEAD} Ahead: ", batch_xs[LOOK_AHEAD-1].shape)
    print(f"{ROI_names[-1]} Train Batch y Shape for {LOOK_AHEAD} Ahead: ", batch_ys[LOOK_AHEAD-1].shape)

TP Train Batch X Shape for 1 Ahead:  (20, 188, 1)
TP Train Batch y Shape for 1 Ahead:  (20, 188, 1)
TP Train Batch X Shape for 2 Ahead:  (20, 187, 1)
TP Train Batch y Shape for 2 Ahead:  (20, 187, 1)
TP Train Batch X Shape for 3 Ahead:  (20, 186, 1)
TP Train Batch y Shape for 3 Ahead:  (20, 186, 1)
TP Train Batch X Shape for 4 Ahead:  (20, 185, 1)
TP Train Batch y Shape for 4 Ahead:  (20, 185, 1)
TP Train Batch X Shape for 5 Ahead:  (20, 184, 1)
TP Train Batch y Shape for 5 Ahead:  (20, 184, 1)
TP Train Batch X Shape for 6 Ahead:  (20, 183, 1)
TP Train Batch y Shape for 6 Ahead:  (20, 183, 1)
TP Train Batch X Shape for 7 Ahead:  (20, 182, 1)
TP Train Batch y Shape for 7 Ahead:  (20, 182, 1)
TP Train Batch X Shape for 8 Ahead:  (20, 181, 1)
TP Train Batch y Shape for 8 Ahead:  (20, 181, 1)
TP Train Batch X Shape for 9 Ahead:  (20, 180, 1)
TP Train Batch y Shape for 9 Ahead:  (20, 180, 1)
TP Train Batch X Shape for 10 Ahead:  (20, 179, 1)
TP Train Batch y Shape for 10 Ahead:  (20, 179, 1

In [82]:
batch_xs, batch_ys = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    batch = next(test_data_generator[LOOK_AHEAD-1][-1].generate())
    batch_xs.append(batch[0])
    batch_ys.append(batch[1])
    print(f"{ROI_names[-1]} Test Batch X Shape for {LOOK_AHEAD} Ahead: ", batch_xs[LOOK_AHEAD-1].shape)
    print(f"{ROI_names[-1]} Test Batch y Shape for {LOOK_AHEAD} Ahead: ", batch_ys[LOOK_AHEAD-1].shape)

TP Test Batch X Shape for 1 Ahead:  (10, 188, 1)
TP Test Batch y Shape for 1 Ahead:  (10, 188, 1)
TP Test Batch X Shape for 2 Ahead:  (10, 187, 1)
TP Test Batch y Shape for 2 Ahead:  (10, 187, 1)
TP Test Batch X Shape for 3 Ahead:  (10, 186, 1)
TP Test Batch y Shape for 3 Ahead:  (10, 186, 1)
TP Test Batch X Shape for 4 Ahead:  (10, 185, 1)
TP Test Batch y Shape for 4 Ahead:  (10, 185, 1)
TP Test Batch X Shape for 5 Ahead:  (10, 184, 1)
TP Test Batch y Shape for 5 Ahead:  (10, 184, 1)
TP Test Batch X Shape for 6 Ahead:  (10, 183, 1)
TP Test Batch y Shape for 6 Ahead:  (10, 183, 1)
TP Test Batch X Shape for 7 Ahead:  (10, 182, 1)
TP Test Batch y Shape for 7 Ahead:  (10, 182, 1)
TP Test Batch X Shape for 8 Ahead:  (10, 181, 1)
TP Test Batch y Shape for 8 Ahead:  (10, 181, 1)
TP Test Batch X Shape for 9 Ahead:  (10, 180, 1)
TP Test Batch y Shape for 9 Ahead:  (10, 180, 1)
TP Test Batch X Shape for 10 Ahead:  (10, 179, 1)
TP Test Batch y Shape for 10 Ahead:  (10, 179, 1)
TP Test Batch X Sh

### Building Model

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 2`
- `hidden units = 32`
- `dropout = 0.3`

In [83]:
HIDDEN_UNITS = 32
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 2

In [84]:
def avg_bold_model():
  model = Sequential()
  for _ in range(LSTM_LAYERS):
    model.add(LSTM(HIDDEN_UNITS, return_sequences=True))
  model.add(Dropout(0.3))
  for _ in range(DENSE_LAYERS-1):
    model.add(TimeDistributed(Dense(HIDDEN_UNITS, activation='relu')))
  model.add(TimeDistributed(Dense(1)))
  return model

In [85]:
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***{LOOK_AHEAD} Look Ahead Model***")
    sample_model = avg_bold_model()
    sample_model.build(input_shape=batch_xs[LOOK_AHEAD-1].shape)
    sample_model.summary()

***1 Look Ahead Model***
Model: "sequential_315"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_315 (LSTM)             (10, 188, 32)             4352      
                                                                 
 dropout_315 (Dropout)       (10, 188, 32)             0         
                                                                 
 time_distributed_630 (TimeD  (10, 188, 32)            1056      
 istributed)                                                     
                                                                 
 time_distributed_631 (TimeD  (10, 188, 1)             33        
 istributed)                                                     
                                                                 
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________
***2 Look Ahead Model***


                                                                 
 time_distributed_644 (TimeD  (10, 181, 32)            1056      
 istributed)                                                     
                                                                 
 time_distributed_645 (TimeD  (10, 181, 1)             33        
 istributed)                                                     
                                                                 
Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________
***9 Look Ahead Model***
Model: "sequential_323"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_323 (LSTM)             (10, 180, 32)             4352      
                                                                 
 dropout_323 (Dropout)       (10, 180, 32)             0         
                         

Total params: 5,441
Trainable params: 5,441
Non-trainable params: 0
_________________________________________________________________


In [86]:
models = [[avg_bold_model() for _ in ROI_names] for _ in LOOK_AHEADS]

#### Training Model

If you haven't already trained the models then uncomment and train your models which will be saved and will be loaded for inference later.

In [87]:
for i in range(len(LOOK_AHEADS)):
    for j in range(len(ROI_names)):
        models[i][j].compile(optimizer = 'adam', loss = 'mean_squared_error')

In [88]:
EPOCHS = 100
checkpoint_filepaths = [[f"./Models/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{name}/model_weights-best-val-loss-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{name}" for name in ROI_names] for LOOK_AHEAD in LOOK_AHEADS]
model_checkpoint_callbacks = [[tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepat,
    save_weights_only=True,
    monitor = "val_loss",
    mode='min',
    save_best_only=True) for checkpoint_filepat in checkpoint_filepath] for checkpoint_filepath in checkpoint_filepaths]

In [89]:
def plot_train_test_loss(history, model_name, i):  
  plt.figure(figsize=(10,10))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title(model_name+' loss')
  plt.ylabel('loss')
  plt.xlabel('epochs')
  plt.legend(['train', 'val'], loc='upper left')
  plt.savefig("./Plots/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/"+ROI_names[i]+"/"+model_name+"-loss.png")

In [ ]:
histories = list()
with tf.device('/device:GPU:0'):
    for LOOK_AHEAD in LOOK_AHEADS:
        history = list()
        for j in range(len(ROI_names)):
            hist = models[LOOK_AHEAD-1][j].fit(
                        train_data_generator[LOOK_AHEAD-1][j].generate(),
                        steps_per_epoch=train_data[i].shape[0]//TRAIN_BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=test_data_generator[LOOK_AHEAD-1][j].generate(),
                        validation_steps=test_data[i].shape[0]//TEST_BATCH_SIZE,
                        callbacks=[model_checkpoint_callbacks[LOOK_AHEAD-1][j]]
                        )
            models[LOOK_AHEAD-1][j].save_weights(f"./Models/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[j]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[j]}")
            plot_train_test_loss(hist, f"{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[j]}", j)
            history.append(hist)
        histories.append(history)

In [ ]:
histories = list()
with tf.device('/device:GPU:0'):
    for LOOK_AHEAD in LOOK_AHEADS:
        history = list()
        for j in range(len(ROI_names)):
            hist = models[LOOK_AHEAD-1][j].fit(
                        train_data_generator[LOOK_AHEAD-1][j].generate(),
                        steps_per_epoch=train_data[i].shape[0]//TRAIN_BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=test_data_generator[LOOK_AHEAD-1][j].generate(),
                        validation_steps=test_data[i].shape[0]//TEST_BATCH_SIZE,
                        callbacks=[model_checkpoint_callbacks[LOOK_AHEAD-1][j]]
                        )
            models[LOOK_AHEAD-1][j].save_weights(f"./Models/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[j]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[j]}")
            plot_train_test_loss(hist, f"{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[j]}", j)
            history.append(hist)
        histories.append(history)

In [106]:
for LOOK_AHEAD in LOOK_AHEADS:
    for i in range(len(ROI_names)):
        models[LOOK_AHEAD-1][i].load_weights(f"./Models/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [ ]:
preds = [[models[LOOK_AHEAD-1][i].predict(next(test_data_generator[LOOK_AHEAD-1][i].generate())[0]) for i in range(len(ROI_names))] for LOOK_AHEAD in LOOK_AHEADS]

In [109]:
a_TRs = np.array([56, 67, 73, 81, 134, 142, 155, 167, 174, 180])
d_TRs = np.array([67, 72, 85, 89, 145, 154, 168, 174, 180, 188])
and_TRs = np.union1d(a_TRs, d_TRs)

def highlight_and(act, pred):
  min_y_list = list()
  max_y_list = list()
  for i in range(5):
    min_y_list.append(np.min(np.concatenate([act[a_TRs[2*i]:d_TRs[2*i+1]], pred[a_TRs[2*i]:d_TRs[2*i+1]]])))
    max_y_list.append(np.max(np.concatenate([act[a_TRs[2*i]:d_TRs[2*i+1]], pred[a_TRs[2*i]:d_TRs[2*i+1]]])))
    # plt.fill_betweenx(y=[min_y_list[i], max_y_list[i]], x1=a_TRs[2*i], x2=d_TRs[2*i+1], color='gray', alpha=0.1)
    plt.fill_betweenx(y=[min_y_list[i], max_y_list[i]], x1=a_TRs[2*i], x2=a_TRs[2*i+1], color='violet', alpha=0.4)
    plt.fill_betweenx(y=[min_y_list[i], max_y_list[i]], x1=d_TRs[2*i], x2=d_TRs[2*i+1], color='greenyellow', alpha=0.4)

In [ ]:
for LOOK_AHEAD in LOOK_AHEADS:
    for i in range(len(ROI_names)):
      
        actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[LOOK_AHEAD-1][i][:,:,0], preds[LOOK_AHEAD-1][i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
        plt.figure(figsize=(24, 18))

        for j in range(NUM_TEST_SUBS-1):
  
            ax = plt.subplot(3, 3, j + 1)
            plt.plot(actual_test_data_output[j,:])
            plt.plot(pred_test_data_output[j,:])
            # highlight_and(actual_test_data_output[j,:], pred_test_data_output[j,:])
            plt.xticks(np.concatenate([np.array([0]), and_TRs - LOOK_AHEAD]), np.concatenate([np.array([LOOK_AHEAD]), and_TRs]))
            plt.ylabel("Average BOLD")
            plt.xlabel("TRs")
            plt.title(f"Avg BOLD Preds of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead for {j+1}th Test Sub's {ROI_names[i]}")
            plt.legend(['actual', 'pred'])
        plt.savefig(f"./Plots/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-Preds.png")

In [111]:
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***Each Test Subject's Correlation Coefficient between {LOOK_AHEAD} ahead predicted and actual average BOLD values for ROIs***")
    print()
    avg_r = list()
    for i in range(len(ROI_names)):
        actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[LOOK_AHEAD-1][i][:,:,0], preds[LOOK_AHEAD-1][i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
        for j in range(NUM_TEST_SUBS):
            print(f"{ROI_names[i]} for {j+1}th sub: {np.round(np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1], 2)}")
    print()

    print(f"***Average across test subjects Correlation Coefficient between {LOOK_AHEAD} ahead predicted and actual average BOLD values for ROIs***")
    print()
    for i in range(len(ROI_names)):
        actual_test_data_output = np.squeeze(test_data_unscaled[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[LOOK_AHEAD-1][i][:,:,0], preds[LOOK_AHEAD-1][i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
        sum_r = 0
        for j in range(NUM_TEST_SUBS):
            sum_r += np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1]
        print(f"{ROI_names[i]}: {np.round(sum_r / NUM_TEST_SUBS, 2)}")

***Each Test Subject's Correlation Coefficient between 1 ahead predicted and actual average BOLD values for ROIs***

dlPFC for 1th sub: 0.32
dlPFC for 2th sub: 0.34
dlPFC for 3th sub: 0.46
dlPFC for 4th sub: 0.31
dlPFC for 5th sub: 0.25
dlPFC for 6th sub: 0.15
dlPFC for 7th sub: 0.24
dlPFC for 8th sub: 0.08
dlPFC for 9th sub: 0.32
dlPFC for 10th sub: 0.45
INS for 1th sub: 0.4
INS for 2th sub: 0.82
INS for 3th sub: 0.61
INS for 4th sub: 0.51
INS for 5th sub: -0.06
INS for 6th sub: 0.45
INS for 7th sub: 0.0
INS for 8th sub: 0.54
INS for 9th sub: 0.04
INS for 10th sub: 0.36
VC for 1th sub: 0.57
VC for 2th sub: 0.51
VC for 3th sub: 0.57
VC for 4th sub: 0.66
VC for 5th sub: 0.66
VC for 6th sub: 0.37
VC for 7th sub: 0.41
VC for 8th sub: 0.65
VC for 9th sub: 0.55
VC for 10th sub: 0.59
Cau for 1th sub: 0.18
Cau for 2th sub: 0.65
Cau for 3th sub: 0.39
Cau for 4th sub: 0.16
Cau for 5th sub: 0.39
Cau for 6th sub: 0.1
Cau for 7th sub: 0.01
Cau for 8th sub: 0.27
Cau for 9th sub: 0.68
Cau for 10th s

PCC: 0.13
PCG: 0.11
mCC: 0.11
mOFC: 0.38
Prec: 0.18
mPFC: 0.19
pSTS: 0.11
vlPFC: 0.05
dmPFC: 0.13
Nac: 0.22
IPL: -0.03
ACC: 0.22
lOFC: 0.25
Put: 0.15
AMY: 0.25
TP: 0.36
***Each Test Subject's Correlation Coefficient between 15 ahead predicted and actual average BOLD values for ROIs***

dlPFC for 1th sub: -0.1
dlPFC for 2th sub: 0.01
dlPFC for 3th sub: -0.26
dlPFC for 4th sub: -0.16
dlPFC for 5th sub: 0.25
dlPFC for 6th sub: -0.05
dlPFC for 7th sub: -0.09
dlPFC for 8th sub: -0.19
dlPFC for 9th sub: -0.05
dlPFC for 10th sub: -0.02
INS for 1th sub: -0.04
INS for 2th sub: 0.55
INS for 3th sub: -0.14
INS for 4th sub: 0.28
INS for 5th sub: 0.1
INS for 6th sub: 0.39
INS for 7th sub: -0.11
INS for 8th sub: 0.13
INS for 9th sub: -0.16
INS for 10th sub: 0.2
VC for 1th sub: 0.01
VC for 2th sub: -0.08
VC for 3th sub: -0.27
VC for 4th sub: -0.12
VC for 5th sub: -0.01
VC for 6th sub: 0.0
VC for 7th sub: -0.07
VC for 8th sub: 0.3
VC for 9th sub: 0.04
VC for 10th sub: 0.3
Cau for 1th sub: 0.04
Cau for

In [102]:
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***Model's Final MSE(Train) Loss between {LOOK_AHEAD} ahead predicted and actual average BOLD values for ROIs for the Train Subjects***")
    print()
    for i in range(len(ROI_names)):
        print(ROI_names[i]+":", np.round(histories[LOOK_AHEAD-1][i].history['loss'][-1], 5))
    print()
    print(f"***Model's Final MSE(Validation) Loss between {LOOK_AHEAD} ahead predicted and actual average BOLD values for ROIs for the Test Subjects***")
    print()
    for i in range(len(ROI_names)):
        print(ROI_names[i]+":", np.round(histories[LOOK_AHEAD-1][i].history['val_loss'][-1], 5))

***Model's Final MSE(Train) Loss between 1 ahead predicted and actual average BOLD values for ROIs for the Train Subjects***

dlPFC: 0.00205
INS: 0.00152
VC: 0.0026
Cau: 0.00168
PCC: 0.00195
PCG: 0.00208
mCC: 0.00169
mOFC: 0.00236
Prec: 0.00158
mPFC: 0.00179
pSTS: 0.00154
vlPFC: 0.00162
dmPFC: 0.00226
Nac: 0.00169
IPL: 0.00271
ACC: 0.00164
lOFC: 0.00222
Put: 0.00097
AMY: 0.00115
TP: 0.00181

***Model's Final MSE(Validation) Loss between 1 ahead predicted and actual average BOLD values for ROIs for the Test Subjects***

dlPFC: 0.00099
INS: 0.0011
VC: 0.00364
Cau: 0.00357
PCC: 0.0017
PCG: 0.00291
mCC: 0.00125
mOFC: 0.00191
Prec: 0.00103
mPFC: 0.00233
pSTS: 0.0009
vlPFC: 0.00081
dmPFC: 0.00336
Nac: 0.00279
IPL: 0.00382
ACC: 0.00123
lOFC: 0.00233
Put: 0.00043
AMY: 0.0005
TP: 0.00191
***Model's Final MSE(Train) Loss between 2 ahead predicted and actual average BOLD values for ROIs for the Train Subjects***

dlPFC: 0.00213
INS: 0.00157
VC: 0.00285
Cau: 0.00175
PCC: 0.00213
PCG: 0.00224
mCC: 

In [112]:
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***Model's Final MSE(Validation) Loss between {LOOK_AHEAD} ahead predicted and actual avg voxel BOLD values for ROIs for the Test Subjects***")
    print()
    for i in range(len(ROI_names)):
        actual_test_data_output = np.squeeze(test_data_unscaled[i][:,LOOK_AHEAD:,:])
        pred_test_data_output = scalers[i].inverse_transform(np.hstack([preds[LOOK_AHEAD-1][i][:,:,0], preds[LOOK_AHEAD-1][i][:,:LOOK_AHEAD,0]]))[:,:-LOOK_AHEAD]
        print(ROI_names[i]+" for Each Validation Subject:", tf.keras.losses.mean_squared_error(actual_test_data_output, pred_test_data_output).numpy())
        print(ROI_names[i]+" :", np.mean(tf.keras.metrics.mean_squared_error(actual_test_data_output, pred_test_data_output), axis=-1))

***Model's Final MSE(Validation) Loss between 1 ahead predicted and actual avg voxel BOLD values for ROIs for the Test Subjects***

dlPFC for Each Validation Subject: [1.0584755  0.5694468  1.462727   0.6301395  0.38565704 0.54777944
 0.48806486 1.3279246  1.3587765  0.34313932]
dlPFC : 0.81721306
INS for Each Validation Subject: [0.72528195 0.3515749  0.61163384 0.56636435 1.7652274  1.0888273
 2.0304165  0.7674871  1.3084185  0.81470364]
INS : 1.0029936
VC for Each Validation Subject: [0.71770823 0.3301804  1.1491773  0.39451954 0.34719676 1.1092607
 0.5474221  0.81601924 0.8241601  0.8342295 ]
VC : 0.7069874
Cau for Each Validation Subject: [1.4051586 2.077672  1.7618742 3.4349618 4.0508122 4.4538536 2.264484
 7.5210238 1.1385103 2.3105168]
Cau : 3.0418866
PCC for Each Validation Subject: [0.80196184 0.54721403 0.8462801  0.72360265 1.877292   1.963073
 0.93332213 1.3731712  1.2948407  0.5153254 ]
PCC : 1.0876083
PCG for Each Validation Subject: [0.2933377  0.90092605 1.0234485  0.4

Put : 1.9374771
AMY for Each Validation Subject: [2.136174  1.584614  2.5776184 1.7248307 1.6515746 2.16494   1.5771943
 7.6841063 2.9721985 3.0843349]
AMY : 2.7157588
TP for Each Validation Subject: [ 6.8327637  5.6671147  2.982439   1.6927172  9.139405   8.316456
  2.309421  18.84515    2.0088916 10.735138 ]
TP : 6.8529496
***Model's Final MSE(Validation) Loss between 4 ahead predicted and actual avg voxel BOLD values for ROIs for the Test Subjects***

dlPFC for Each Validation Subject: [1.3446827  2.0176568  1.1584696  0.9259452  0.83307993 1.1100458
 1.0551995  0.82905054 3.204693   1.0166494 ]
dlPFC : 1.3495471
INS for Each Validation Subject: [1.3227417 1.4247948 1.4202522 1.2874781 1.2969676 7.30232   1.5705074
 6.638874  1.7057697 6.766606 ]
INS : 3.0736313
VC for Each Validation Subject: [5.2441564 0.8470522 1.9702204 1.7058913 1.1830035 1.3007504 0.9818538
 7.1837964 2.9913554 7.1957746]
VC : 3.0603855
Cau for Each Validation Subject: [ 6.3971014 12.081916   2.3942244 21.1714

ACC : 3.0798705
lOFC for Each Validation Subject: [ 3.608662   4.5875063  1.5331033  2.0577977  3.2645376  1.8196186
  1.5692525  3.5520165  3.3408866 20.867495 ]
lOFC : 4.6200876
Put for Each Validation Subject: [1.1328089  0.7049701  1.1697384  0.7824327  2.5109415  1.3615775
 0.95110595 8.828021   0.8873793  1.5728877 ]
Put : 1.9901861
AMY for Each Validation Subject: [2.8090887 1.2850028 2.6367748 1.7360954 1.7909201 2.3260422 1.7241899
 6.4243097 2.9008663 2.1565735]
AMY : 2.5789864
TP for Each Validation Subject: [ 7.82837    6.1869926  3.1772332  1.9072932 10.698299   9.643876
  2.4510992 22.892002   2.303883  12.497785 ]
TP : 7.958684
***Model's Final MSE(Validation) Loss between 7 ahead predicted and actual avg voxel BOLD values for ROIs for the Test Subjects***

dlPFC for Each Validation Subject: [1.5103582  1.9419043  1.4192678  0.8943842  0.7105079  0.92451555
 0.9659432  0.8932162  2.8969889  0.9764709 ]
dlPFC : 1.3133557
INS for Each Validation Subject: [1.3092332 1.33096

mCC for Each Validation Subject: [0.9888072  1.7035923  2.014987   1.1293088  2.063716   7.9096737
 0.93800706 7.7923017  1.4171277  9.158146  ]
mCC : 3.5115666
mOFC for Each Validation Subject: [2.3596098 1.97189   6.6909513 2.9595718 3.4836345 1.5319252 2.6411567
 9.730127  5.829929  8.577272 ]
mOFC : 4.577607
Prec for Each Validation Subject: [ 2.243671   2.7444777  3.1476123  1.5224489  2.571911  11.23253
  1.3300517  4.791899   3.254533   3.1662245]
Prec : 3.6005359
mPFC for Each Validation Subject: [10.133242   1.0324173  8.300628   2.7396843 10.562443   3.205201
  3.158035  23.010813   7.1694093 13.402226 ]
mPFC : 8.27141
pSTS for Each Validation Subject: [ 2.1737607  1.6425856  3.3043072  5.579575   2.3754396  3.4061263
  2.1964717  3.8733013  1.7851346 11.791326 ]
pSTS : 3.8128028
vlPFC for Each Validation Subject: [1.6548959  0.94221306 2.6424613  1.2836765  0.9674022  3.3956208
 1.6862203  3.8502388  1.8174888  5.784207  ]
vlPFC : 2.4024425
dmPFC for Each Validation Subject:

PCG : 3.0881498
mCC for Each Validation Subject: [0.96880156 1.597526   1.8534253  1.3026091  2.0111666  7.864327
 1.0603946  7.5710063  1.4016905  8.724919  ]
mCC : 3.4355865
mOFC for Each Validation Subject: [2.300144  2.0686126 7.9274898 3.7109466 4.069983  1.3773963 2.67531
 8.563878  7.7531295 7.2095823]
mOFC : 4.765647
Prec for Each Validation Subject: [ 2.2265055  2.3607826  3.1297894  1.6342533  2.480625  10.198661
  1.5856632  4.342526   3.2715824  2.8049505]
Prec : 3.4035335
mPFC for Each Validation Subject: [ 9.737119   1.1257428 10.629234   2.2676258  9.917521   2.7563584
  2.8146503 21.168375   6.5950313 12.40179  ]
mPFC : 7.9413443
pSTS for Each Validation Subject: [ 2.1855514  1.7350602  3.8089004  5.6046114  2.1763415  3.4231434
  2.32973    3.7701502  1.8020897 10.691194 ]
pSTS : 3.7526772
vlPFC for Each Validation Subject: [1.6703633  0.94002867 3.0587714  1.4135382  0.93376774 3.4702668
 1.768908   3.8489418  2.0323782  5.451442  ]
vlPFC : 2.4588408
dmPFC for Each Va

Prec for Each Validation Subject: [ 1.6853132  2.9144764  2.8321562  1.3750615  2.1182399 10.5515785
  1.3863902  4.799329   3.3848982  3.0749304]
Prec : 3.4122376
mPFC for Each Validation Subject: [ 9.122826   1.0328261 11.2052     2.014311   9.417793   2.1849205
  2.3892078 21.028328   5.9873047 11.619122 ]
mPFC : 7.6001844
pSTS for Each Validation Subject: [ 2.0394192  1.5093912  3.7525403  5.653873   1.9007003  3.6179452
  2.2143657  4.0499926  1.6703107 11.164565 ]
pSTS : 3.7573104
vlPFC for Each Validation Subject: [1.502245  0.8923022 3.0035849 1.3040462 0.8025085 3.0925336 1.7317643
 3.5282423 2.1666574 4.9603925]
vlPFC : 2.2984276
dmPFC for Each Validation Subject: [ 3.7430978  8.848813   4.5244875  2.0575783  1.0691458 17.985277
  3.1932855  1.3534791 10.925425  18.529926 ]
dmPFC : 7.2230515
Nac for Each Validation Subject: [ 2.4980488  5.808038   3.1037037 13.733378   1.8391079 13.366584
  1.6967256 27.436321   2.560652   5.3506713]
Nac : 7.7393227
IPL for Each Validation Su